In [ ]:
import scala.collection.mutable
import scala.math.{pow, sqrt}
import smile.plot._
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.math.distances.ContinuousDistance
import org.clustering4ever.clustering.epsilonproximity.rdd.EpsilonProximityScalar
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.clusterizables.EasyClusterizable
import org.apache.spark.storage.StorageLevel
import org.clustering4ever.vectors.ScalarVector
import org.clustering4ever.hashing.LDLSH
import org.clustering4ever.spark.clustering.GradientAscent
import org.clustering4ever.shapeless.{VMapping, VectorizationMapping}
import org.clustering4ever.clustering.indices.MultiExternalIndicesLocal

import scala.collection.mutable
import scala.math.{pow, sqrt}
import smile.plot._
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.math.distances.ContinuousDistance
import org.clustering4ever.clustering.epsilonproximity.rdd.EpsilonProximityScalar
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.clusterizables.EasyClusterizable
import org.apache.spark.storage.StorageLevel
import org.clustering4ever.vectors.ScalarVector
import org.clustering4ever.hashing.LDLSH
import org.clustering4ever.spark.clustering.GradientAscent
import org.clustering4ever.shapeless.{VMapping, VectorizationMapping}
import org.clustering4ever.clustering.indices.MultiExternalIndicesLocal


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/DS1/DS1.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/DS1/DS1-labels.csv

--2019-05-09 18:17:11--  http://www.clustering4ever.org/Datasets/DS1/DS1.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73786 (72K) [text/csv]
Saving to: ‘/tmp/DS1.csv.19’

     0K .......... .......... .......... .......... .......... 69% 9.29M 0s
    50K .......... .......... ..                              100% 11.3M=0.007s

2019-05-09 18:17:11 (9.82 MB/s) - ‘/tmp/DS1.csv.19’ saved [73786/73786]

:sh: Scheme missing.
--2019-05-09 18:17:11--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2019-05-09 18:17:11--  http://www.clustering4ever.org/Datasets/DS1/DS1-labels.csv
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 19315 (19K) [text/csv]
Saving to: ‘/tmp/DS1-labels.cs

In [ ]:
type SeqType = mutable.ArrayBuffer[Double]
val path = "/tmp/DS1.csv"
val data = scala.io.Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).zipWithIndex
  .map{ case (v, id) => EasyClusterizable(id.toLong, ScalarVector(v)) }.toSeq
val rdd = sc.parallelize(data)
val labelsPath = "/tmp/DS1-labels.csv"

defined type alias SeqType
path: String = /tmp/DS1.csv
data: Seq[org.clustering4ever.clusterizables.EasyClusterizable[None.type,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]]] = Stream(EasyClusterizable(0,Vectorizable(None),ScalarVector(ArrayBuffer(122.0, 199.0)),shapeless.HMap@393972e3,Vector()), ?)
rdd: org.apache.spark.rdd.RDD[org.clustering4ever.clusterizables.EasyClusterizable[None.type,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]]] = ParallelCollectionRDD[0] at parallelize at <console>:87
labelsPath: String = /tmp/DS1-labels.csv


## Gradient Ascent & Epsilon Proximity commons parameters

In [ ]:
val bucketNumber = 12 // Approximatively DS_Size / 1000 for decent performance, the large buckets are the slower it becomes
val bucketLayers = 1 // Neighbor bucket layer take into account during Gradient Ascent, at least 1 is recommended for decent k values, the bigger it is the slower it become

val persistanceLVL: StorageLevel = StorageLevel.MEMORY_ONLY // Persistance Spark level 
val dim = 2 // Dim of your dataset
val w = 1D // Don't observe any change by changing this value, let it to whatever you want
val lsh = LDLSH[SeqType](dim, w)
val trialsNumber = 10

bucketNumber: Int = 12
bucketLayers: Int = 1
persistanceLVL: org.apache.spark.storage.StorageLevel = StorageLevel(memory, deserialized, 1 replicas)
dim: Int = 2
w: Double = 1.0
lsh: org.clustering4ever.hashing.LDLSH[SeqType] = LDLSH(2,1.0)
trialsNumber: Int = 10


## Gradient Ascent parameters

In [ ]:
val k = 30 // Main Gradient Ascent parameter, the k of knn gradient ascent
val k2 = 1 // Optional parameter which determine if studied point has to switch to its knn majority vote neighbor bucket // STAY FIX OVER XP
val epsilon = 0.01 // Stopping criteria for Gradient Ascent, every point have to moove less than epsilon in order to stop iteration // STAY FIX OVER XP
val maxNumberIter = 20 // STAY FIX OVER XP
val propConvStopIter = 1D // Supplementary stopping criteria, stop if propConvStopIter % of data have converged // STAY FIX OVER XP
val memoryExpensive = true // Choose for fast or slow computation at some memory cost
// (building of similarity matrix per bucket per iter or on the fly (more calculus)) // STAY FIX OVER XP

k: Int = 30
k2: Int = 1
epsilon: Double = 0.01
maxNumberIter: Int = 20
propConvStopIter: Double = 1.0
memoryExpensive: Boolean = true


## Epsilon proximity parameters

In [ ]:
// Main parameter epsilon can be approximate with average distance knn => "knn:88" or be expressed directly as a double "eps:3.67"
val epsilonEpsProx = "knn:100"
// val epsilonEpsProx = "eps:2.56"
val classicalMetric = Euclidean[SeqType](false) // STAY FIX OVER XP
val methodChoice = "bydot:1" // Fusionning cluster methods, at least X dot have to satisfy fusionning condition with "bydot:X", X=1 is greatly recommended // STAY FIX OVER XP
val divisionFactor = 1 // Experimental, let it to 1, it is supposed to speedup process on large datasets with bigger values but doesn't affect results // STAY FIX OVER XP
val cmin = 0 // Fusion clusters that have less than X elements with their closest cluster, it is in O(c²) with c number of clusters. // STAY FIX OVER XP

epsilonEpsProx: String = knn:100
classicalMetric: org.clustering4ever.math.distances.scalar.Euclidean[SeqType] = Euclidean distance without squared root applied
methodChoice: String = bydot:1
divisionFactor: Int = 1
cmin: Int = 0


In [ ]:
// Theses 3 params have to be here else there is some bugs..., it works more properly on full programm versions
val idOriginalVector = 8
val idToStoreGAVector = 0
val vm = VMapping[Int, ScalarVector[SeqType]]

val trials = (0 until trialsNumber).map{ _ =>
  val t0 = System.currentTimeMillis

  val ascended = GradientAscent.train(rdd, k, epsilon, maxNumberIter, bucketNumber, bucketLayers, k2, propConvStopIter, memoryExpensive,
    persistanceLVL, idToStoreGAVector, true)


  val updatedData = ascended.map( cz => cz.addAlternativeVector(idOriginalVector, cz.v).switchForExistingVectorization(idToStoreGAVector, vm) )

  val model = EpsilonProximityScalar(
    epsilonEpsProx,
    methodChoice,
    classicalMetric,
    bucketNumber,
    lsh,
    cmin,
    bucketLayers,
    StorageLevel.MEMORY_ONLY,
    divisionFactor
  ).fit(updatedData)

  val clusterizedRDD = model.obtainInputDataClustering(updatedData)

  val predLabels = clusterizedRDD.map(_.clusterIDs.head).collect

  val t1 = System.currentTimeMillis

  val trueLabels = scala.io.Source.fromFile(labelsPath).getLines.map(_.toInt).toSeq.toArray
  val targetAndPred = trueLabels.zip(predLabels)

  val mei = new MultiExternalIndicesLocal(targetAndPred)

  val nmi = mei.nmiSQRT
  val rand = mei.rand

  val duration = (t1 - t0) / 1000D
  (nmi, rand, duration)
}



idOriginalVector: Int = 8
idToStoreGAVector: Int = 0
vm: org.clustering4ever.shapeless.VMapping[Int,org.clustering4ever.vectors.ScalarVector[SeqType]] = org.clustering4ever.shapeless.VMapping@72deca56
trials: scala.collection.immutable.IndexedSeq[(Double, Double, Double)] = Vector((0.0032867822618504483,0.7073869127703936,70.38), (0.0034345907801531313,0.7076585192366903,74.256), (0.0033763581551431326,0.7076375088911963,76.841), (0.003816431146950281,0.7089513239955718,76.932), (0.0032865836524378607,0.7073788428876925,76.421), (0.0035237955893258044,0.7086001647211086,78.33), (0.003286583652438072,0.7073788428876925,74.371), (0.003286583652438072,0.7073788428876925,79.59), (0.003487155587370376,0.7086311549807125,80.319), (0.002865570363003344,0.7070291161367858,77.376))


In [ ]:
val (nmiSum, randSum, durationSum) = trials.reduce{ (a, b) => (a._1 + b._1, a._2 + b._2, a._3 + b._3) }
val nmiMean = nmiSum / trials.size
val randMean = randSum / trials.size
val durationMean = durationSum / trials.size

val (nmiSDnoSqrtnoDivided, randSDnoSqrtnoDivided, durationSDnoSqrtnoDivided) = trials
  .map{ case (nmi, rand, duration) => (pow(nmi - nmiMean, 2), pow(rand - randMean, 2), pow(duration - durationMean, 2)) }
  .reduce{ (a, b) => (a._1 + b._1, a._2 + b._2, a._3 + b._3) }

val nmiSD = sqrt(nmiSDnoSqrtnoDivided / trials.size)
val randSD = sqrt(randSDnoSqrtnoDivided / trials.size)
val durationSD = sqrt(durationSDnoSqrtnoDivided / trials.size)

nmiSum: Double = 0.03365043484111052
randSum: Double = 7.078031229395536
durationSum: Double = 764.8159999999999
nmiMean: Double = 0.0033650434841110523
randMean: Double = 0.7078031229395536
durationMean: Double = 76.48159999999999
nmiSDnoSqrtnoDivided: Double = 5.228953201806108E-7
randSDnoSqrtnoDivided: Double = 3.999974694531574E-6
durationSDnoSqrtnoDivided: Double = 75.57747440000011
nmiSD: Double = 2.286690447307223E-4
randSD: Double = 6.324535314575747E-4
durationSD: Double = 2.7491357623806087


In [ ]:
println(
  "nmi : " + nmiMean + " +- " + nmiSD + "\n" +
  "rand : " + randMean + " +- " + randSD + "\n" +
  "duration : " + durationMean + " +- " + durationSD
)

nmi : 0.0033650434841110523 +- 2.286690447307223E-4
rand : 0.7078031229395536 +- 6.324535314575747E-4
duration : 76.48159999999999 +- 2.7491357623806087
